# Praktikum Pengantar Pembelajaran Mesin


---
## Bab 7. Support Vector Machine (SVM) Lanjutan


Tuliskan Nama, NIM, dan kelas Anda:

Nama  : Dani Adrian

NIM   : 225150201111009

Kelas : TIF-E

### 1) Import Data

Unduh dataset yang akan digunakan pada praktikum kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

In [93]:
! wget https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv

--2024-05-04 11:56:44--  https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3975 (3.9K) [text/plain]
Saving to: ‘iris.csv.5’

iris.csv.5          100%[===================>]   3.88K  --.-KB/s    in 0s      

2024-05-04 11:56:44 (33.4 MB/s) - ‘iris.csv.5’ saved [3975/3975]



Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu


In [94]:
import pandas as pd
import numpy as np
data = pd.read_csv('iris.csv')



Cek isi dataset Anda dengan menggunakan perintah **head()**

In [95]:
data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**.


In [96]:
from sklearn.model_selection import train_test_split
data_latih,data_uji = train_test_split(data,test_size=0.2)

Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 120 data, dan **data_uji** terdiri dari 30 data

In [97]:
print(data_uji.shape[0])
print(data_latih.shape[0])

30
120


Pisahkan label/kelas dari data uji menjadi sebuah variabel bernama **label_uji**

In [98]:
label_uji = data_uji.pop('variety')

## 3) Pembentukan data latih one-vs-rest

Metode one-vs-rest memerlukan tiga jenis data latih yang diperlukan untuk melatih tiga SVM yang berbeda pada dataset Iris. Fungsi **buat_trainingset** digunakan untuk membentuk tiga dataset tersebut.

In [99]:
def buat_trainingset(dataset):
  trainingset = {}
  kolom_kelas = dataset.columns[-1]
  list_kelas = dataset[kolom_kelas].unique()
  for kelas in list_kelas:
    data_temp = dataset.copy(deep=True)
    data_temp[kolom_kelas]=data_temp[kolom_kelas].map({kelas:1})
    data_temp[kolom_kelas]=data_temp[kolom_kelas].fillna(-1)
    trainingset[kelas]=data_temp
  return trainingset

Gunakan fungsi **buat_trainingset** untuk membentuk data latih dengan nama variabel **trainingset** yang akan digunakan pada proses training.

In [100]:
trainingset = buat_trainingset(data_latih)

Tampilkan isi **trainingset** agar Anda dapat memahami struktur dari variabel tersebut.

In [101]:
print(trainingset)

{'Virginica':      sepal.length  sepal.width  petal.length  petal.width  variety
121           5.6          2.8           4.9          2.0      1.0
106           4.9          2.5           4.5          1.7      1.0
94            5.6          2.7           4.2          1.3     -1.0
77            6.7          3.0           5.0          1.7     -1.0
112           6.8          3.0           5.5          2.1      1.0
..            ...          ...           ...          ...      ...
89            5.5          2.5           4.0          1.3     -1.0
63            6.1          2.9           4.7          1.4     -1.0
3             4.6          3.1           1.5          0.2     -1.0
90            5.5          2.6           4.4          1.2     -1.0
104           6.5          3.0           5.8          2.2      1.0

[120 rows x 5 columns], 'Versicolor':      sepal.length  sepal.width  petal.length  petal.width  variety
121           5.6          2.8           4.9          2.0     -1.0
106      

## 4) Pembentukan SVM Biner

Tujuan dari algoritma SVM adalah meminimalkan nilai *cost function*. Penghitungan nilai minimal dapat dapat dilakukan dengan menghitung nilai gradien dari *cost function* terlebih dahulu. Fungsi di bawah ini berguna untuk menghitung nilai gradien cost function

In [102]:
def hitung_cost_gradient(W,X,Y,regularization):
    jarak = 1 - (Y* np.dot(X,W))
    dw = np.zeros(len(W))
    if max(0,jarak)==0:
      di=W
    else:
      di = W - (regularization * Y*X)
    dw += di
    return dw

Terdapat beberapa cara untuk meminimalkan nilai *cost function*, salah satunya menggunakan Stochastic Gradient Descent (SGD) untuk melakukan minimasi. Minimasi *cost function* merupakan inti dari algoritma SVM. Fungsi di bawah ini merupakan implementasi algoritma SGD

In [103]:
from sklearn.utils import shuffle
def sgd(data_latih,label_latih,learning_rate = 0.000001,max_epoch=1000,regularization=10000):
  data_latih = data_latih.to_numpy()
  label_latih = label_latih.to_numpy()
  bobot = np.zeros(data_latih.shape[1])
  for epoch in range(1,max_epoch):
    X,Y = shuffle(data_latih,label_latih,random_state=101)
    for index,x in enumerate(X):
      delta = hitung_cost_gradient(bobot,x,Y[index],regularization)
      bobot = bobot - (learning_rate * delta)
  return bobot

## 5) Proses Training

Proses training dilakukan dengan memanggil fungsi **sgd** berulang kali sesuai banyaknya kelas yang ada pada data. Dengan demikian, proses training menghasilkan bobot sebanyak kelas yang ada pada dataset. Buatlah fungsi bernama **training** yang digunakan untuk melakukan proses training one-vs-rest

In [104]:
def training(trainingset):
  list_kelas = trainingset.keys()
  w = {}
  for kelas in list_kelas:
    data_latih = trainingset[kelas]
    label_latih = data_latih.pop(data_latih.columns[-1])
    w[kelas] = sgd(data_latih,label_latih)
  return w

Lakukan proses training dengan memanggil fungsi **training** dan menempatkan hasilnya pada variabel **W**

In [105]:
W = training(trainingset)

Tampilkan isi variabel **W**

In [106]:
print(W)

{'Virginica': array([-4.00892872, -4.76987115,  5.96302083,  6.39008762]), 'Versicolor': array([ 0.65295868, -2.10326134,  1.51718483, -4.39252812]), 'Setosa': array([ 0.23219148,  0.68035152, -1.1158642 , -0.51021062])}


## 6) Proses *testing* biner
Proses testing dilakukan dengan menghitung nilai [*dot product*](https://en.wikipedia.org/wiki/Dot_product) antara bobot hasil training dengan data uji. Kelas data ditentukan berdasarkan tanda (positif atau negatif) dari hasil dot product tersebut. Fungsi berikut mengimplementasikan proses testing

In [107]:
def testing(W,data_uji):
  prediksi = np.array([])
  for i in range(data_uji.shape[0]):
    y_prediksi = np.sign(np.dot(W,data_uji.to_numpy()[i]))
    prediksi = np.append(prediksi,y_prediksi)
  return prediksi

## TUGAS
Pada tugas kali ini Anda mendefinisikan proses testing pada metode one-vs-rest. Proses testing pada metode one-vs-rest dilakukan dengan memanggil proses testing biner untuk setiap **value** pada dictionary **W**. Kelas pada sebuah data latih adalah **key** pada dictionary **W** yang memiliki nilai prediksi **1**. Lengkapi fungsi **testing_onevsrest** di bawah ini. Output dari fungsi tersebut adalah list nama kelas hasil prediksi.

In [108]:
def testing_onevsrest(W,data_uji):
  #tulis kode Anda di sini.
  hasil = pd.DataFrame(columns=W.keys())
  print(hasil)
  for kelas in W.keys():
    hasil[kelas] = testing(W[kelas],data_uji)
  kelas_prediksi = hasil.idxmax(1)
  print(kelas_prediksi)
  return kelas_prediksi

In [109]:
prediksi = testing_onevsrest(W, data_uji)

Empty DataFrame
Columns: [Virginica, Versicolor, Setosa]
Index: []
0         Setosa
1      Virginica
2      Virginica
3     Versicolor
4      Virginica
5         Setosa
6         Setosa
7      Virginica
8      Virginica
9      Virginica
10        Setosa
11     Virginica
12     Virginica
13     Virginica
14     Virginica
15        Setosa
16     Virginica
17        Setosa
18        Setosa
19     Virginica
20     Virginica
21     Virginica
22        Setosa
23     Virginica
24     Virginica
25     Virginica
26     Virginica
27     Virginica
28     Virginica
29        Setosa
dtype: object


Berapa banyak data latih yang berhasil diprediksi dengan benar?

In [110]:
print(sum(prediksi==label_uji.to_numpy()))

21
